# Model Define

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda, Input, GlobalAveragePooling1D
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [30]:
class Custom_Model() :
    def __init__(self, weight, input_shape) :
        self.checkpoint = weight
        self.model = self.build_model(input_shape)
        
    def build_model(self, input_shape : tuple):
        input = Input(shape=input_shape)
        x = LSTM(128, return_sequences=True, activation='tanh', dropout=0.2)(input)
        x = LSTM(64, return_sequences=True, activation='tanh', dropout=0.2)(x)
        x = LSTM(32, return_sequences=True, activation='tanh', dropout=0.2)(x)
        x = GlobalAveragePooling1D()(x)
        output = Dense(1)(x)
        return Model(input, output)
        
    
    def load_model(self) :
        self.model.load_weights(self.checkpoint)
        return self.model


In [34]:
window_size = 6
n_feature = 6

weight = './checkpoints/ckeckpointer.ckpt'
input_shape = (window_size, n_feature)

model = Custom_Model(weight, input_shape)
lstm_model = model.load_model()
lstm_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 6, 6)]            0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 6, 128)            69120     
_________________________________________________________________
lstm_22 (LSTM)               (None, 6, 64)             49408     
_________________________________________________________________
lstm_23 (LSTM)               (None, 6, 32)             12416     
_________________________________________________________________
global_average_pooling1d_7 ( (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 130,977
Trainable params: 130,977
Non-trainable params: 0
_____________________________________________________

# Inference Data Preprocess

In [35]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pyupbit

In [ ]:
class Data_preprocess() :
    def __init__(self, ticker, interval, to, count) :
        dataset = pyupbit.get_ohlcv(ticker=ticker, interval=interval, to=to, count=count)